In [1]:
import requests
from requests import get
from bs4 import BeautifulSoup as bs
import re
import pandas as pd
import numpy as np
from tqdm import tqdm

import datetime

from time import sleep
from random import randint

In [2]:
url = "https://www.offiziellecharts.de/charts/single/for-date-1515106800000"

cw = 0
y = 2018

while True:
    rank_current = []
    artist = []
    title = []
    label = []
    incharts = []
    peak = []
    
    page = requests.get(url)
    soup = bs(page.content, parser="html5lib")
    
    #finding out the start and end date of the top 100 songs list
    periodhelp = soup.find_all('strong')
    
    weekdates = []
    for dt in periodhelp:
        weekdates.append(dt.text)
    #getting list of current rank
    
    rankc_help = soup.find_all('span', {'class' : 'this-week'})
    rank_current = [int(r.text) for r in rankc_help]
    
    #getting list of artists
    artist_help = soup.find_all('span', {'class' : 'info-artist'})
    artist = [r.text for r in artist_help]
    
    #getting list of titles
    title_help = soup.find_all('span', {'class' : 'info-title'})
    title = [r.text for r in title_help]
    
    #getting list of labels
    label_help = soup.find_all('span', {'class' : 'info-label'})
    label = [r.text for r in label_help]
    
    #getting list of in charts information
    
    incharts_help = soup.find_all('span', {'class' : 'plus-data'})
    incharts = [r.text.split()[2] for r in incharts_help if len(r.text.split()) > 2]
    
    #getting list of overall peak position information
    peak_help = soup.find_all('span', {'class' : 'plus-data'})
    peak = [r.text.split()[1] for r in peak_help if len(r.text.split()) < 3]
    
    dict_help = {'current_rank': rank_current,
                 'artist': artist,
                 'title': title,
                 'label': label,
                 'in_charts': incharts,
                 'peak': peak,
                }
    
    df = pd.DataFrame(dict_help)
    df['weekbegin'] = datetime.datetime.strptime(weekdates[0], "%d.%m.%Y")
    df['weekend'] = datetime.datetime.strptime(weekdates[1], "%d.%m.%Y")
    df['year'] = datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").year
    df['month'] = datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").month
    
    if int(datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").year) == y:
        cw += 1
        savename_pickle = "../pickle/charts_" + str(y) + "_" + str(cw) + ".pkl"
        savename_csv = "../csv/charts_" + str(y) + "_" + str(cw) + ".csv"
        df.to_pickle(savename_pickle)
        df.to_csv(savename_csv, index=False)
        
    elif int(datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").year) > y:
        y = int(datetime.datetime.strptime(weekdates[0], "%d.%m.%Y").year)
        cw = 1
        savename_pickle = "../pickle/charts_" + str(y) + "_" + str(cw) + ".pkl"
        savename_csv = "../csv/charts_" + str(y) + "_" + str(cw) + ".csv"
        df.to_pickle(savename_pickle)
        df.to_csv(savename_csv, index=False)
    
    if len(soup.find_all('a', {'class' : 'next-link btn btn-default btn-sm'})) > 0:
        url = "https://www.offiziellecharts.de" + str([a['href'] for a in soup.find_all('a', {'class' : 'next-link btn btn-default btn-sm'})][0]) + "/"
        #Nap time!
        wait_time = randint(1,5)
        sleep(wait_time)
        continue
    
    elif (len(soup.find_all('a', {'class' : 'next-link btn btn-default btn-sm'})) <= 0):
        break